# Overview of relational data

## The basic relation (i.e. the table)

The term technical term “relation” can be interchanged with the standard notion that we have of “tabular data,” for instance a “Person” relation

| ID | Last Name | First Name | Role |
| :---: | :---: | :---: | :---: |
| 1 | Kolter | Zico | Instructor |
| 2 | Xi | Edgar | TA |
| 3 | Lee | Mark | TA |
| 4 | Mani | Shouvik | TA |
| 5 | Gates | Bill | Student |
| 6 | Musk | Elon | Student |

**Rows are called tuples (or records)**, represent a single instance of this relation, and must be unique.

while **Columns are called attributes**, specify some elements contained by each of the tuples.

**A primary key is a unique ID** for every tuple in a relation (i.e. every row in the table), each relation must have exactly one primary key.

**A foreign key is an attribute** that points to the primary key of another relation. If you delete a primary key, need to delete all foreign keys pointing to it

# Indexes (not indices)
Indexes are created as ways to *“quickly”* access elements of a table. For example, consider finding people with last name “Gates”: no option but just scan through the whole dataset: $O(n)$ operation.

Think of an index as a separate sorted table containing the indexed column and the tuple location: searching for value takes $O(log  n)$ time

The primary key always has an index associated with it (so you can think of primary keys themselves as always being a fast way to access data). Indexes don’t have to be on a single column, can have an index over multiple columns (with some ordering)

# Entity Relationships

Several types of inter-table relationships
1. One-to-one
2. (One-to-zero/one)
3. One-to-many (and many-to-one)
4. Many-to-many

These relates one (or more) rows in a table with one (or more) rows in another table, via a foreign key. Note that these relationships are really between the “entities” that the tables represent.

### One-to-many Relationship

We have already seen a one-to-many relationship: one role can be shared by many people, denoted as follows

![One-to-many relationship.](one_to_many.svg)

### One-to-one relationships

In a true one-to-one relationship spanning multiple tables, each row in a table has exactly one row in another table. Not very common to break these across multiple tables, as you may as well just add another attribute to an existing table, but it is possible:

![One-to-one relationship.](one_to_one.svg)

### One-to-zero/one relationships

More common in databases is to find “one-to-zero/one” relationships broken across multiple tables. Consider adding a “Grades” table to be our database: each person can have at most one tuple in the grades table:

![One-to-zero/one relationship.](one_to_zero_one.svg)

Bars and circles denote “mandatory” versus “option” relationships (we won’t worry about these, just know that there is notation for them)

### Many-to-many relationships

Creating a grades table as done before is a bit cumbersome, because we need to keep adding columns to the table, null entries if someone doesn’t do the homework. Alternatively, consider adding two tables, a “homework” table that represents
information about each homework, and an associative table that links homeworks
to people:



### Associative tables

<center>**Person Homework**</center>

| Person ID | HW ID | Score |
| :---: | :---: | :---: |
| 5 | 1 |100 | 2018-02-07 |
| 5 | 2 | 80 | 2018-02-21 |
| 6 | 1 | 60 | 2018-02-21 |
| 6 | 2 | 80 | 2018-02-21 |

1. What is the primary key of this table? 
2. What are foreign keys?
a.Person ID
b. HW ID
c. (Person ID, HW ID)
d. (Person ID, HW ID, Score)
3. Which indexes would you want to create on this table?

### Many-to-many relationships


Setups like this encode many-to-many relationships: each person can have multiple homeworks, and each homework can be done by multiple people:

![many-to-many relationship.](many_to_many.svg)

We could also write this in terms of relationships specified by the associative table, but this is not really correct, as it is mixing up the underlying relationships with how they are stored in a database:

![An alternative possibility for the many-to-many relationship.](many_to_many2.svg)

# Pandas

Pandas is a “Data Frame” library in Python, meant for manipulating in-memory data with row and column labels (as opposed to, e.g., matrices, that have no row or column labels). Pandas is not a relational database system, but it contains functions that mirror some functionality of relational databases.

As mentioned, Pandas is not a relational data system, in particular it has no notion of primary keys (but it does have indexes). Operations in Pandas are typically not in place (that is, they return a new modified DataFrame, rather than modifying an existing one).

Use the “inplace” flag to make them done in place. If you select a single row or column in a Pandas DataFrame, this will return a “Series” object, which is like a one-dimensional DataFrame (it has only an index and corresponding values, not multiple columns)

In [18]:
import pandas as pd

df = pd.DataFrame([(1, 'Kolter', 'Zico'), 
                   (2, 'Xi', 'Edgar'),
                   (3, 'Lee', 'Mark'), 
                   (4, 'Mani', 'Shouvik'),
                   (5, 'Gates', 'Bill'),
                   (6, 'Musk', 'Elon')], 
                  columns=["id", "last_name", "first_name"])
df

#or 



# df = [(1, 'Kolter', 'Zico'), 
#                    (2, 'Xi', 'Edgar'),
#                    (3, 'Lee', 'Mark'), 
#                    (4, 'Mani', 'Shouvik'),
#                    (5, 'Gates', 'Bill'),
#                    (6, 'Musk', 'Elon')]
# pd.DataFrame(df, columns=["id", "last_name", "first_name"])

# this second method does not allow the operation of the set_index method. It may also prevent the operation of other 
# methods because df wasn't assigned to the dataframe. the set_index method is a panda dataframe method not a panda
# pdSeries method

,id,last_name,first_name
0,1,Kolter,Zico
1,2,Xi,Edgar
2,3,Lee,Mark
3,4,Mani,Shouvik
4,5,Gates,Bill
5,6,Musk,Elon


The first column in the displayed data frame is the "index" of each row. Unfortunately the terminology here clashes with database terminology, but "index" for Pandas actually means something more like "primary key" in a database table (though with the exception that it is possible to have duplicate entries, though it is recommended to avoid this in Pandas). 

That is, an index (if done right, without duplicate indices) is a identifier for each row in the database. We can set the index to one of the existing columns using the .set_index() call.


In [19]:
df.set_index("id")

,last_name,first_name
id,,
1,Kolter,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon


But you need to be very careful about one thing here. By default, most Pandas operations, like .set_index() and many others, and not done in place. That is, while the df.set_index("id") call above returns a copy of the df dataframe with the index set to the id column (remember that Jupyter notebook displays the return value of the last line in a cell), the original df object is actually unchanged here:

In [20]:
df

,id,last_name,first_name
0,1,Kolter,Zico
1,2,Xi,Edgar
2,3,Lee,Mark
3,4,Mani,Shouvik
4,5,Gates,Bill
5,6,Musk,Elon


If we want to actually change the df object itself, you need to use the inplace=True flag for these functions (or assign the original object to the result of a function, but this isn't as clean):

In [21]:
df.set_index("id", inplace=True)
df

,last_name,first_name
id,,
1,Kolter,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon


There are also some peculiarities that you'll get used to with Pandas, such as the fact that if you select a single column or row from a Pandas DataFrame, you don't get a single-column or single-row DataFrame, but an alternative Pandas type called a Series, which is kind of like a 1D data frame (but it has some different indexing rules). There are reasons behind all these design decisions, even if they can seem a bit arbitrary at first.

There are actually a few different ways to access elements in a Pandas DataFrame, the commonly used ones are the .loc or .iloc properties.

### Common Pandas data access

Let's consider a few of the common ways to access or set data in a Pandas DataFrame. You can access individual elements using the **.loc[row, column] notation,** where **row denotes the index you are searching for** and **column denotes the column name.** For example, to access the last name of person with ID 1 we would execute:

In [22]:
df.loc[1, "last_name"]

'Kolter'

In [23]:
# If we want to access all last names, (or all elements in a particular row), we use the : wildcard. For example:

df.loc[:, "last_name"]

id
1    Kolter
2        Xi
3       Lee
4      Mani
5     Gates
6      Musk
Name: last_name, dtype: object

Notice the output here looks a bit different than the nicely-formed typical Pandas output, which stems from the fact (mentioned previously), that this returned object is actually not a Pandas DataFrame, but a Pandas Series. If we do want to get a DataFrame with just this one column, we can get a "list" of columns (with just one elements)

In [24]:
df.loc[:, ["last_name"]]

,last_name
id,
1,Kolter
2,Xi
3,Lee
4,Mani
5,Gates
6,Musk


which now gives us the desired 1D data frame. We can do a similar thing with row indexes.

In [25]:
df.loc[[1,2], :]

,last_name,first_name
id,,
1,Kolter,Zico
2,Xi,Edgar


which will select only a subject of the allowable rows.

We can additionally use .loc to change the content of existing entries:

In [26]:
df.loc[1, "last_name"] = "Kitler"
df

,last_name,first_name
id,,
1,Kitler,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon


We can even add additional rows/columns that don't exist.

In [28]:
df.loc[7, :] = ("moore", "zoe")
df

,last_name,first_name
id,,
1,Kitler,Zico
2,Xi,Edgar
3,Lee,Mark
4,Mani,Shouvik
5,Gates,Bill
6,Musk,Elon
7,moore,zoe


Finally, remember that .loc always indexes based upon the "index" (i.e., effectively primary key) of the data frame along with the column name. If you want to instead access based upon positional index (i.e., using 0-indexed counters for both the rows and columns), you can use the .iloc property

In [29]:
df.iloc[4] # to get the info for this row

last_name     Gates
first_name     Bill
Name: 5, dtype: object

In [30]:
df.iloc[4,1] # to get just the first_name for this row. 

'Bill'

In [ ]:
# read CSV file into DataFrame
df = pd.read_csv(filename)
# get first five rows of DataFrame
df.head()
# index into a dataframe
# df.loc[rows, columns] and df.iloc[row numbers, column numbers]
df.loc[:, "Last Name"] # Series of all last names
df.loc[:, ["Last Name"]] # DataFrame with one column
df.loc[[1,2], :] # DataFrame with only ids 1,2
df.loc[1,"Last Name"] = "Kilter" # Set an entry in a DataFrame
df.loc[7,:] = ("Moore", "Andrew") # Add a new entry with index=7
df.iloc[0,0] 

# SQLite

1. An actual relational database management system (RDBMS). 
2. Unlike most systems, it is a serverless model, applications directly connect to a file
3. Allows for simultaneous connections from many applications to the same database file (but not quite as much concurrency as client-server systems)
5. All operations in SQLite will use SQL (Structured Query Language) command issued to the database object
6. You can enforce foreign keys in SQLite.

Unlike Pandas, SQLite actually is a full-featured database, but unlike most production databases, it does not use a client/server model. Databases are instead stored direclty on disk and accessed just via the library. 

This has the advantage of being very simple, with no server to configure and run, but for large applications it is typically insufficient: because files are not very good at concurrent access (that is, many different processes/threads cannot simultaneously read and write from a single file), the system is not ideal for very large databases where multiple threads need to be constant readings from and writing to the database. Note that SQLite does have some limited forms of concurrency in this respect, but nothing sophisticated when compared to larger scale databases. If you do want to investigate a more "production strength" client/server database, you can check out PostreSQL.

SQLite, as the name suggests, uses the SQL (structured query language) language for interacting with the database; note both "Sequel" and "Ess Queue Ell" are acceptable prononciations of SQL.

Interacting with SQLite (or any other SQL-based database) from Python is not ideal, because you typically use Python code to generate SQL expressions as strings, then execute them, which is not the most beautiful coding paradigm. For simple databases, though, it usually suffices to get the job done.

Let's look at how to create a simple database with the "Person" and "Grades" tables that we had considered earlier.

In [100]:
import sqlite3
conn = sqlite3.connect("happy.db")
cursor = conn.cursor()

# conn.close() closing this database while in use produces this error:
# ProgrammingError: Cannot operate on a closed database.

This code imports the library, creates a connection to the "database.db" file (it will create it if it does not already exist), and then creates a "cursor" into the database. The notion of cursor is common to a lot of database libraries, but essentially a cursor is an object that allows us to interact with the database. 

If we want to create the Person and Grades tables we saw above (to keep things simple, and later to illustrate joins, we'll use the first version of the Grades table, with no associative table), we would use the following syntax:

In [108]:
cursor.execute( """
CREATE TABLE Person(
id INTEGER PRIMARY KEY,
last_name TEXT,
first_name TEXT
);""")


cursor.execute ("""
CREATE TABLE Grades(
id INTEGER PRIMARY KEY,
hw1_grade INTEGER,
    hw2_grade INTEGER
);

""")
conn.commit()

OperationalError: table Person already exists

Let's insert some data into these tables. The syntax for this operation, hopefully fairly straightforward is given by the following:

In [109]:
# cursor.execute("INSERT INTO person VALUES (1, 'Kolter', 'Zico');")
# I get this error: IntegrityError: UNIQUE constraint failed: Person.id. It only goes away when I comment the first 
# line of code out. I've observed that once you run a cell with db values, you can't run them more than once.
# I have to change the id values entered previously.
cursor.execute("INSERT INTO person VALUES (13, 'Lu', 'Weise');")
cursor.execute("INSERT INTO person VALUES (14, 'Jun', 'Lin');")
cursor.execute("INSERT INTO person VALUES (15, 'Moore', 'Adams');")
cursor.execute("INSERT INTO person VALUES (16, 'Emma', 'Bridge');")
cursor.execute("INSERT INTO person VALUES (17, 'Tommy', 'Sea');")

cursor.execute("INSERT INTO grades VALUES (18, 80, 90);")
cursor.execute("INSERT INTO grades VALUES (19, 85, 65);")
cursor.execute("INSERT INTO grades VALUES (170, 120, 200);")

[Note that I'm an additional row to the grades tables with person_id=100 for illustration purposes later when we talk about joins. Notice that this actually would violate the consistency of the data if we had required grades.person_id to be a true foreign key, but since we didn't bother, it works fine]. If we want to see what has been added to the database, we can do this the "SQLite Python" way, which involves running a query and then iterating over the rows in a result returned by a cursor.execute() result, as so

In [110]:
for row in cursor.execute("SELECT * FROM Person;"):
    print(row)

(13, 'Lu', 'Weise')
(14, 'Jun', 'Lin')
(15, 'Moore', 'Adams')
(16, 'Emma', 'Bridge')
(17, 'Tommy', 'Sea')


The row data outputted above (and also below) is more than what was typed into the cell where the values were created in a previous cell beacause you cannot run a cell more than once at creation of values for your database.

In [111]:
for row in cursor.execute("SELECT * FROM Grades;"):
    print(row)

(18, 80, 90)
(19, 85, 65)
(170, 120, 200)


Alternatively, it can be handy to dump the results of a query directly into a Pandas DataFrame. Fortunately, Pandas provides a nice call for doing this, the pd.read_sql_query() function, with takes the database connection and an optional argument to set the index of the Pandas dataframe to be one of the columns:

In [112]:
pd.read_sql_query("SELECT * FROM Grades;", conn)

,id,hw1_grade,hw2_grade
0,18,80,90
1,19,85,65
2,170,120,200


In [113]:
pd.read_sql_query("SELECT * FROM Person;", conn, index_col="id")

,last_name,first_name
id,,
13,Lu,Weise
14,Jun,Lin
15,Moore,Adams
16,Emma,Bridge
17,Tommy,Sea


In [114]:
# compare both codes
pd.read_sql_query("SELECT * FROM Person;", conn)

,id,last_name,first_name
0,13,Lu,Weise
1,14,Jun,Lin
2,15,Moore,Adams
3,16,Emma,Bridge
4,17,Tommy,Sea


The SELECT statement is probably the SQL command you'll use most in data science: it is used to query data from th database. We won't here go into a full description of the SELECT statement, except to say that the most common syntax is something like

<columns> here will be a comma-separated list of all the columns to select, or the wildcard * to denote all columns. <tables> is a comma-separated list of tables. And <conditions> is a list of conditions, typically separated by AND if there are multiple conditions, that specify what subset to return. For example, let's see how to select all the last name (and id) from the persons table with id > 2

In [115]:
pd.read_sql_query("SELECT id,last_name FROM person WHERE id > 2;", conn, index_col="id")

,last_name
id,
13,Lu
14,Jun
15,Moore
16,Emma
17,Tommy


we can also delete values from tables using the DELETE FROM SQL command, using a similar WHERE clause as in the SELECT command.

In [116]:
cursor.execute("INSERT INTO person VALUES (7, 'Moore', 'Andrew');")
pd.read_sql_query("SELECT * from person;", conn, index_col="id")

,last_name,first_name
id,,
7,Moore,Andrew
13,Lu,Weise
14,Jun,Lin
15,Moore,Adams
16,Emma,Bridge
17,Tommy,Sea


In [117]:
cursor.execute("DELETE FROM person where id = 7;")
pd.read_sql_query("SELECT * from person;", conn, index_col="id")      

,last_name,first_name
id,,
13,Lu,Weise
14,Jun,Lin
15,Moore,Adams
16,Emma,Bridge
17,Tommy,Sea


# Joins

Join puts operations multiple tables into a single relation, matching between attributes in the two tables. There are four types of joins, though only the first two are used much in practice:

- Inner
* Left
+ Right
- Outer

You join two tables on columns from each table, where these columns specify how to match the rows between the two columns. This should become more clear with a few examples. In the examples that follow, we're going to consider our Person and Grades tables, that we just created above, and we're join to join the tables on the person.id and grades.person_id columns.

### Inner joins

If you don't know what type of join you want, you probably want an inner join.  This does the "obvious" thing, of only returning those rows where the two columns in each table have matching values, and it appends the rows together for each of these matching rows.  In our case, an inner join between the Person and Grades table would return the following:

| ID | Last Name | First Name | HW1 Grade | HW2 Grade |
| :---: | :---: | :---: | :---: | :---: |
| 5 | Gates | Bill | 85 | 95 |
| 6 | Musk | Elon | 80 | 60 |

Let's see how this can be done programatically, first in Pandas and then in SQL.  In Pandas, you should do joins with the `.merge()` command: there is an alternative `.join()` command, but this always assumes you want to join on the index column for one of the data frames, and _not_ the index frame for another, and overall is just a special case of `.merge()` so I prefer to learn the generic call.

In [124]:
df_Person = pd.read_sql_query("SELECT * FROM Person", conn)
df_Grades = pd.read_sql_query("SELECT * FROM Grades", conn)
df_Person.merge(df_Grades, how="inner", left_on = "id", right_on="id")

,id,last_name,first_name,hw1_grade,hw2_grade


Hopefully the syntax of .merge() looks straightforward: you call the method on the "left" data frame, and pass the "right" data frame as the first argument. The how parameter specifies the type of join (here inner), and the left_on and right_on arguments specify the column name that you want to join on for the left and right data frames respectively. If you alternatively want to join on the index for the left or right data frame, you specify the left_index or right_index parameters as so:

In [125]:
df_Person.set_index("id", inplace=True)
df_Grades.set_index("id", inplace=True)

In [126]:
df_Person.merge(df_Grades, how="inner", left_index=True, right_index=True)

,last_name,first_name,hw1_grade,hw2_grade
id,,,,


This call is maybe a bit cleaner, as it doesn't include the (duplicate) columns you match on as in the previous example, but this is a minor difference.

In SQL, an inner join is specified by the WHERE clause in a SELECT statement.

In [121]:
pd.read_sql_query("SELECT * FROM Person, Grades WHERE Person.id = Grades.id" , conn)

,id,last_name,first_name,id,hw1_grade,hw2_grade


Exactly the same behavior as our Pandas join, but the advantage of course is that for very complex joins and large database queries, a true database will be faster at performing queries like this than the Pandas code.

### Left joins

Whereas an inner join only kept those rows with corresponding entires in both tables, a left join will keep all the items in the left table, and add in the attribution from the right table (filling with NaNs if no match exists in the right table). Any row value that occurs in the right table but not the left table is discarded.

In [123]:
df_person = pd.read_sql_query("SELECT * FROM Person", conn)
df_grades = pd.read_sql_query("SELECT * FROM Grades", conn)
df_person.merge(df_grades, how="left", left_on = "id", right_on="id")

,id,last_name,first_name,hw1_grade,hw2_grade
0,13,Lu,Weise,NaN,NaN
1,14,Jun,Lin,NaN,NaN
2,15,Moore,Adams,NaN,NaN
3,16,Emma,Bridge,NaN,NaN
4,17,Tommy,Sea,NaN,NaN


### Right joins

A right join does what you might expect, the converse of the left join, where all the rows in the right matrix are kept. While SQLite has no syntax for right joins (you can achieve the same results by simply reversing the order of the two tables and doing a left join), Pandas does have built-in syntax for the right join

In [127]:
df_person.merge(df_grades, how="right", left_on = "id", right_on="id")

,id,last_name,first_name,hw1_grade,hw2_grade
0,18,NaN,NaN,80,90
1,19,NaN,NaN,85,65
2,170,NaN,NaN,120,200


### Outer joins

Finally, outer joins (also called a cross product) do what you may expect, and keep all rows that occur in either table, so essentially take the union of the left and right joins. Again, SQLite has no syntax for it (you can achieve the same thing via a UNION statement, but we won't cover it), but Pandas has the function.

In [128]:
df_person.merge(df_grades, how="outer", left_on = "id", right_on="id")

,id,last_name,first_name,hw1_grade,hw2_grade
0,13,Lu,Weise,NaN,NaN
1,14,Jun,Lin,NaN,NaN
2,15,Moore,Adams,NaN,NaN
3,16,Emma,Bridge,NaN,NaN
4,17,Tommy,Sea,NaN,NaN
5,18,NaN,NaN,80.0,90.0
6,19,NaN,NaN,85.0,65.0
7,170,NaN,NaN,120.0,200.0


While you will probably use inner joins the vast majority of the time (and left joins the remainder), it is helpful to understand the different types of join operations from a conceptual standpoint, and how they all fit together